In [ ]:
import numpy as np
import cupy as cp

In [ ]:
x = np.array(([1,2,3],[4,5,6]))
x

In [ ]:
x_gpu = cp.array(x)
type(x_gpu)


In [ ]:
x_gpu * x_gpu

In [ ]:
sigma = cp.ones((784,10))
mu = cp.ones((784,10))
n_mc_samples = 1000

In [ ]:
def __softmax(x,y): # needs self in model
    # sort observations
    print("{} {}".format("len(y)", len(x.shape)))
    if len(x.shape) != 2:
        # if only one observation array is given, reshape it
        # only one ftr should not happen...
        x = x.reshape(1,len(x))
 
    observed_classes = np.unique(y)

    for obs_class in observed_classes:
        # fix access
        print("obs_class: {}".format(obs_class))
        observations_index = np.where(y == obs_class)[0]
        print(observations_index)
        x_obs = x[observations_index]
        print(x_obs)
        n_obs = len(x_obs)
        print(n_obs)
        x_obs = cp.array(x_obs)
    
        for epoch in range(epochs): #changed to self.epoch in model
                
                # Iterative update of mu and sigma
                try:
                    # o number of obs, l number of samples, j features, c classes
                    # create 3d array with all r for current observation for multiple observation calculation we would need 4d array
                    # r^cl_j = r[l, j, c] lxjxc
                    r = cp.random.randn(n_obs, no_mc_samples, n_total_ftr, len(target_values))
                    #r = np.array([[[-0.558, 1.555], [0.325, -0.726], [0.347,-0.159]],
                    #              [[-0.955, 0.283], [0.115,-1.637], [-0.516,0.161]]])  
                    print(r)
                    # we only change the psi for the actuall given class still need all classes of course
                    # r = np.random.randn(monte_carlo, n_total_ftr)

                    print(r.shape)
                    # calculate thetas for all samples and classes theta^cl_jt = theta[l,j,c]
                    # oxlxjxc
                    theta = r * sigma + mu
                    print(theta.shape)
                    print(theta)
                    #calculate all the etas
                    eta = cp.einsum("oljc,oj->oljc", theta, x_obs) # multiply all ftr_cols with given ftr_vector x
                    print('theta * x:')
                    print(eta)
                    eta = cp.einsum("oljc->olc", eta) #sum up all theta^cl_j * x_tj so we got l samples for all c classes
                    print("eta:")
                    print(eta)
                    eta = cp.exp(eta) # we only need them exp
                    print("eta_exp:")
                    print(eta)
                    print(eta.shape)
                    eta_sum = cp.einsum("olc->ol", eta) #sum up etas for the l samples
                    print("eta_sum:")
                    print(eta_sum)
                    print(eta_sum.shape)
                    #calculate softmax only for observed class
                    #observation_etas = eta[:,y] with more observations we need transposition
                    obs_eta = eta[:,:,obs_class]
                    print("Obs_eta shape: {}".format(obs_eta.shape))
                    softmax_lh = obs_eta / eta_sum # 
                    print(softmax_lh)
                    print(softmax_lh.shape) #should be oxl
                    #marginal = np.einsum("lo->o", softmax_lh) / monte_carlo # 1xy
                    marginal = cp.einsum("ol->o", softmax_lh) / no_mc_samples
                    print(marginal.shape) #should be o
                    print(marginal)
                    # calculate derivatives nabla_mu, nabla_sigma must be handled better

                    #first calculate softmax dtheta
                    # x_eta means observations x times the beloning etas
                    x_eta = cp.einsum("oj,ol->olj", x_obs, obs_eta)
                    print(x_eta)

                    softmax_derivative = cp.einsum("olj,ol->olj", x_eta, (eta_sum - obs_eta))
                    softmax_derivative = cp.einsum("olj->jol", softmax_derivative) /  eta_sum**2
                    softmax_derivative = cp.einsum("jol->olj", softmax_derivative)
                    print(softmax_derivative.shape)
                    print(softmax_derivative)
                    nabla_mu = cp.einsum("olj->oj", softmax_derivative) / no_mc_samples
                    print(nabla_mu.shape) #oj
                    print(nabla_mu)
                    r_jc = r[:,:,:,obs_class]
                    print(r_jc.shape)
                    nabla_sigma = cp.einsum("olj->oj", softmax_derivative * r_jc) / no_mc_samples
                    print(nabla_sigma.shape) #oj
                    # Update parameters
                    mu[:,obs_class] += lr_mu * cp.einsum("jo->j", (nabla_mu.T / marginal))
                    sigma[:,obs_class] += lr_sigma * cp.einsum("jo->j",(nabla_sigma.T / marginal))
                    

                except TypeError as e:
                        raise TypeError('All features must be a numeric data type.') from e